In [11]:
import numpy as np
import pandas as pd
import re, string, random, pickle, nltk, scipy

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from collections import Counter

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk import FreqDist
from nltk.classify import ClassifierI
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import word_tokenize
from IPython.core.display import clear_output

In [6]:
stop_words = list(set(stopwords.words('english')))
text_train = pd.read_csv(r"datasets/review_text_train.csv", index_col = False, delimiter = ',', header=0)
meta_train = pd.read_csv(r"datasets/review_meta_train.csv", index_col = False, delimiter = ',', header=0)
text_test = pd.read_csv(r"datasets/review_text_test.csv", index_col = False, delimiter = ',', header=0)

In [10]:
all_reviews1 = []
all_words = []
allowed_p_o_s = ["J","V","R"]
lemmatizer = WordNetLemmatizer()

for i in range(0,text_train.shape[0]):
    print("Requesting page {}/{}".format(i, text_train.shape[0]))
    clear_output(wait = True)
    text = text_train.at[i,'review']
    rating = meta_train.at[i,'rating']
    tup = (text,rating)
    all_reviews1.append(tup)
    
    cleaned_text = re.sub(r'[^(a-zA-Z)\s]',' ', text)
    tokenized_text = word_tokenize(cleaned_text)
    
    text_w_o_stop = [word for word in tokenized_text if not word in stop_words]

    for token, tag in nltk.pos_tag(text_w_o_stop):
        if tag[:2] == "NN":
            p_o_s = 'n'
        elif tag[:2] == 'VB':
            p_o_s = 'v'
        else:
            p_o_s = 'a'
        token_word = lemmatizer.lemmatize(token, p_o_s)
        if tag[0] in allowed_p_o_s:
            all_words.append(token_word.lower())

In [12]:
all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())[:5000]
random.shuffle(all_reviews1)

all_reviews = [a[0] for a in all_reviews1]
all_ratings = [a[1] for a in all_reviews1]

lim = int(len(all_reviews)*.8)
my_reviews_train_clean = all_reviews[:lim]
my_reviews_test_clean = all_reviews[lim:]
my_target_train = all_ratings[:lim]
my_target_test = all_ratings[lim:]

In [14]:
my_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
my_vectorizer.fit(my_reviews_train_clean)
X_all = my_vectorizer.transform(all_reviews)
my_test_reviews = list(text_test['review'])
X_test_all = my_vectorizer.transform(my_test_reviews)

In [15]:
X = my_vectorizer.transform(my_reviews_train_clean)
X_test = my_vectorizer.transform(my_reviews_test_clean)


In [16]:
class EnsembleClassifier(ClassifierI):
    
    def __init__(self, *classifiers):
        self._classifiers = classifiers
    
    def classify(self, given_features):
        classifier_votes = []
        for c in self._classifiers:
            vote = int(c.predict(given_features))
            classifier_votes.append(vote)
        return Counter(classifier_votes).most_common(1)[0][0]


In [25]:
from numpy import linspace

In [99]:
def nb_hyperparameters(model):
    final_model = None
    max_acc = 0
    for fit_prior in [True, False]:
        for alpha in [0.0001, 0.001,0.01]+ list(linspace(0.1,1,10)):
            classifier = model(alpha=alpha, fit_prior=fit_prior)
            final_model, max_acc = optimal_model(classifier, max_acc, final_model)
    print(f'{final_model=}, {max_acc=}')
    return final_model

In [114]:
def optimal_model(classifier, max_acc, final_model):
    classifier.fit(X, my_target_train)
    predictions = classifier.predict(X_test)
    acc = accuracy_score(my_target_test, predictions)
    if acc>max_acc:
        max_acc=acc
        final_model = classifier
    return final_model, max_acc

In [116]:
def lr_hyperparameters(model):
    final_model = None
    max_acc = 0
    i=1
    for fit_intercept in [True, False]:
        for c in [0.0001, 0.001,0.01]+ list(linspace(0.1,1,10)):
            classifier = model(fit_intercept=fit_intercept, C=c)
            final_model, max_acc = optimal_model(classifier, max_acc, final_model)
            print(f'{i}/13')
    print(f'{final_model=}, {max_acc=}')
    return final_model         

In [117]:
def tune_hyper_parameters():
    n_iter = 10
    classifiers = [MultinomialNB, BernoulliNB, LogisticRegression, SGDClassifier, LinearSVC]
    # mnb = nb_hyperparameters(MultinomialNB)
    # bnb = nb_hyperparameters(BernoulliNB)

    lr = lr_hyperparameters(LogisticRegression)
    max_acc = 0
    sgd = None
    max_acc = 0
    lsvc = None
    max_acc = 0
    return lr

In [119]:
a

LogisticRegression()

In [118]:
a = tune_hyper_parameters()

1/13
1/13
1/13


/Users/sabastianbouma/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1/13


/Users/sabastianbouma/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1/13


...

1/13


/Users/sabastianbouma/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1/13


/Users/sabastianbouma/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1/13
final_model=LogisticRegression(), max_acc=0.8847524047025294


In [52]:
n_iter = 11
max_acc = 0
params = {}
model = None
for fit_prior in [True, False]:
    for alpha in linspace(0,1,n_iter):
        MNB_clf = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
        MNB_clf.fit(X, my_target_train)
        predictions = MNB_clf.predict(X_test)
        acc = accuracy_score(my_target_test, predictions)
        print(f'PARAMS: {fit_prior=}, {alpha=} => {acc=}')
        if acc>max_acc:
            max_acc=acc
            params['fit_prior'] = fit_prior
            params['alpha'] = alpha
            model = MNB_clf

    

/Users/sabastianbouma/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


PARAMS: fit_prior=True, alpha=0.0 => acc=0.7299608122550766
PARAMS: fit_prior=True, alpha=0.1 => acc=0.8630210188813681
PARAMS: fit_prior=True, alpha=0.2 => acc=0.8639116494478091
PARAMS: fit_prior=True, alpha=0.30000000000000004 => acc=0.8583897399358746
PARAMS: fit_prior=True, alpha=0.4 => acc=0.848414677591735
PARAMS: fit_prior=True, alpha=0.5 => acc=0.8323833273957962
PARAMS: fit_prior=True, alpha=0.6000000000000001 => acc=0.8120769504809405
PARAMS: fit_prior=True, alpha=0.7000000000000001 => acc=0.7931955824723904
PARAMS: fit_prior=True, alpha=0.8 => acc=0.7732454577841111
PARAMS: fit_prior=True, alpha=0.9 => acc=0.7582828642679017
PARAMS: fit_prior=True, alpha=1.0 => acc=0.7442109013181333


/Users/sabastianbouma/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


PARAMS: fit_prior=False, alpha=0.0 => acc=0.7308514428215176
PARAMS: fit_prior=False, alpha=0.1 => acc=0.8639116494478091
PARAMS: fit_prior=False, alpha=0.2 => acc=0.8648022800142501
PARAMS: fit_prior=False, alpha=0.30000000000000004 => acc=0.8603491271820449
PARAMS: fit_prior=False, alpha=0.4 => acc=0.8517990737442109
PARAMS: fit_prior=False, alpha=0.5 => acc=0.8375489846811542
PARAMS: fit_prior=False, alpha=0.6000000000000001 => acc=0.8188457427858924
PARAMS: fit_prior=False, alpha=0.7000000000000001 => acc=0.799608122550766
PARAMS: fit_prior=False, alpha=0.8 => acc=0.7830423940149626
PARAMS: fit_prior=False, alpha=0.9 => acc=0.7641610260064126
PARAMS: fit_prior=False, alpha=1.0 => acc=0.7529390808692554


In [31]:
params

{'fit_prior': False, 'alpha': 0.2}

In [18]:
MNB_clf = MultinomialNB(alpha=0.5, fit_prior=False)
MNB_clf.fit(X, my_target_train)
model = MNB_clf
name = str(model).split('(')[0]
predictions = model.predict(X_test)
print ("Accuracy for %s model = %s" % (name, accuracy_score(my_target_test, predictions)))
print(precision_recall_fscore_support(my_target_test, predictions))
print(confusion_matrix(my_target_test, predictions))
print('\n')

Accuracy for MultinomialNB model = 0.8375489846811542
(array([0.92215569, 0.75315126, 0.85228031]), array([0.3255814 , 0.5614722 , 0.99145963]), array([0.48125   , 0.64333782, 0.91661682]), array([ 473, 1277, 3864]))
[[ 154  204  115]
 [  11  717  549]
 [   2   31 3831]]




In [17]:
#results based on training set
MNB_clf = MultinomialNB(alpha=0.5, fit_prior=False)
MNB_clf.fit(X, my_target_train)

BNB_clf = BernoulliNB(alpha = 0.001)
BNB_clf.fit(X, my_target_train)

LogReg_clf = LogisticRegression(C=0.5)
LogReg_clf.fit(X, my_target_train)

SGD_clf = SGDClassifier(loss='perceptron')
SGD_clf.fit(X, my_target_train)

SVC_clf = LinearSVC(C=0.05)
SVC_clf.fit(X, my_target_train)

classifiers = [MNB_clf, BNB_clf, LogReg_clf, SGD_clf, SVC_clf]
for model in classifiers:
    name = str(model).split('(')[0]
    predictions = model.predict(X_test)
    print ("Accuracy for %s model = %s" % (name, accuracy_score(my_target_test, predictions)))
    print(precision_recall_fscore_support(my_target_test, predictions))
    print(confusion_matrix(my_target_test, predictions))
    print('\n')
    
ensemble_clf = EnsembleClassifier(MNB_clf, BNB_clf, LogReg_clf, SGD_clf, SVC_clf)
ensemble_preds = [ensemble_clf.classify(features) for features in X_test]
print ("Accuracy for ensemble model = %s" % (accuracy_score(my_target_test, ensemble_preds)))
print(precision_recall_fscore_support(my_target_test, ensemble_preds))
print(confusion_matrix(my_target_test, ensemble_preds))

print('\n')

/Users/sabastianbouma/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy for MultinomialNB model = 0.8375489846811542
(array([0.92215569, 0.75315126, 0.85228031]), array([0.3255814 , 0.5614722 , 0.99145963]), array([0.48125   , 0.64333782, 0.91661682]), array([ 473, 1277, 3864]))
[[ 154  204  115]
 [  11  717  549]
 [   2   31 3831]]


Accuracy for BernoulliNB model = 0.7907018168863555
(array([0.87709497, 0.75972927, 0.79128819]), array([0.33192389, 0.35160532, 0.99197723]), array([0.48159509, 0.48072805, 0.88033992]), array([ 473, 1277, 3864]))
[[ 157  111  205]
 [  22  449  806]
 [   0   31 3833]]


Accuracy for LogisticRegression model = 0.8843961524759529
(array([0.84061697, 0.78880407, 0.9164607 ]), array([0.69133192, 0.72826938, 0.95962733]), array([0.7587007 , 0.75732899, 0.93754741]), array([ 473, 1277, 3864]))
[[ 327  101   45]
 [  54  930  293]
 [   8  148 3708]]


Accuracy for SGDClassifier model = 0.8815461346633416
(array([0.85714286, 0.78227848, 0.91266913]), array([0.65961945, 0.72592013, 0.96014493]), array([0.74551971, 0.7530463 ,

In [15]:
#results for the competition test set
MNB_clf = MultinomialNB(alpha=0.5, fit_prior=False)
MNB_clf.fit(X_all, all_ratings)

BNB_clf = BernoulliNB(alpha = 0.001)
BNB_clf.fit(X_all, all_ratings)

LogReg_clf = LogisticRegression(C=0.5)
LogReg_clf.fit(X_all, all_ratings)

SGD_clf = SGDClassifier(loss='perceptron')
SGD_clf.fit(X_all, all_ratings)

SVC_clf = LinearSVC(C=0.05)
SVC_clf.fit(X_all, all_ratings)

classifiers = [MNB_clf, BNB_clf, LogReg_clf, SGD_clf, SVC_clf]
for model in classifiers:
    name = str(model).split('(')[0]
    predictions = model.predict(X_test_all)

ensemble_clf = EnsembleClassifier(MNB_clf, BNB_clf, LogReg_clf, SGD_clf, SVC_clf)
ensemble_preds = [ensemble_clf.classify(features) for features in X_test_all]

Accuracy for MultinomialNB model = 0.8254364089775561
(array([0.93333333, 0.71987642, 0.84435016]), array([0.31237323, 0.54102167, 0.98746409]), array([0.46808511, 0.61776403, 0.9103166 ]), array([ 493, 1292, 3829]))
[[ 154  227  112]
 [   8  699  585]
 [   3   45 3781]]


Accuracy for BernoulliNB model = 0.7746704666904168
(array([0.91017964, 0.71153846, 0.7774359 ]), array([0.30831643, 0.31501548, 0.98981457]), array([0.46060606, 0.43669528, 0.87086397]), array([ 493, 1292, 3829]))
[[ 152  128  213]
 [  13  407  872]
 [   2   37 3790]]


Accuracy for LogisticRegression model = 0.8806555040969006
(array([0.86170213, 0.78714524, 0.91014851]), array([0.65720081, 0.72987616, 0.96030295]), array([0.7456847 , 0.75742972, 0.93455331]), array([ 493, 1292, 3829]))
[[ 324  116   53]
 [  39  943  310]
 [  13  139 3677]]


Accuracy for SGDClassifier model = 0.8769148557178482
(array([0.8616188 , 0.74942529, 0.92078451]), array([0.6693712 , 0.75696594, 0.94411073]), array([0.75342466, 0.75317674,

Accuracy for model = 0.8845742785892412
(array([0.9009009 , 0.82309125, 0.89897789]), array([0.65359477, 0.69442262, 0.97424008]), array([0.75757576, 0.75330209, 0.93509705]), array([ 459, 1273, 3882]))


